# 🎓 Введение

Привет, участники соревнования! 👋

Этот ноутбук — ваш проводник в мир анализа временных рядов и построения моделей машинного обучения. Он написан специально для тех, кто только начинает знакомство с Kaggle и участвует в соревнованиях впервые.

Здесь мы вместе:
- 📈 научимся извлекать признаки из временных рядов,
- 🧠 построим базовую модель машинного обучения (LightGBM),
- 🧪 проведём корректную валидацию,
- 📤 подготовим прогноз и отправим его в виде submission-файла.

> 💡 Несмотря на то, что текущее решение не побеждает бенчмарки, **совсем небольшие улучшения** (например, другие признаки, параметры модели или архитектура) уже позволят вам занять **топовые места** в рейтинге!

Цель этого ноутбука — **показать вам базовый, но правильный путь**:
- никаких "магических" трюков,
- только понятные и воспроизводимые шаги.

Удачи в соревновании! 🚀  
**Вы уже на правильном пути.**


In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import os

# 1. Загрузка
work_dir = "../kaggle_data_ts/"
# work_dir = "../input/kazakhstan-ai-respa-take-home/"
train = pd.read_csv(os.path.join(work_dir, "train.csv"))
train['submitted_date'] = pd.to_datetime(train['submitted_date'])
train.head()

/Users/sneddy/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,category,submitted_date,num_papers
0,hep-ph - High Energy Physics - Phenomenology,2000-01-01,5
1,hep-ph - High Energy Physics - Phenomenology,2000-01-02,6
2,hep-ph - High Energy Physics - Phenomenology,2000-01-03,14
3,hep-ph - High Energy Physics - Phenomenology,2000-01-04,10
4,hep-ph - High Energy Physics - Phenomenology,2000-01-05,16


## 📂 Работа с одной категорией

Для начала сосредоточимся на одной подкатегории. Мы извлечём её временной ряд — упорядочим по дате, сделаем дату индексом и уберём лишние колонки. Это поможет нам удобно считать rolling-статистики и готовить признаки.


In [2]:
def get_category_data(df: pd.DataFrame, category: str):
    category_data = df[df['category'] == category].copy()
    category_data = category_data.sort_values('submitted_date')
    category_data = category_data.set_index('submitted_date')
    return category_data.drop('category', axis=1)

category_data = get_category_data(train, 'hep-ph - High Energy Physics - Phenomenology')
category_data

,num_papers
submitted_date,
2000-01-01,5
2000-01-02,6
2000-01-03,14
2000-01-04,10
2000-01-05,16
...,...
2025-02-05,32
2025-02-06,29
2025-02-07,25


## 🔮 Расширение данных на будущее

Чтобы наша модель могла делать прогнозы, мы расширим временной ряд на одну неделю вперёд.  

Мы добавим 7 будущих дат без значений — они нужны, чтобы модель могла "заглянуть в будущее" и сгенерировать признаки для предсказания тестовых данных.

Для простоты мы будем прогнозировать только одну следующую неделю - а дальше мы будем просто повторять наши прогнозы

In [3]:
def extend_dataset(category_data, last_train_date, future_weeks_num: int = 0):
    if future_weeks_num > 0:
        future_dates = pd.date_range(
                start=last_train_date + pd.Timedelta(days=1),
                periods=future_weeks_num * 7,
                freq='D'
            )
            
        future = pd.DataFrame(index= future_dates, data={'num_papers': None})
        return pd.concat((category_data, future))

last_train_date=train.submitted_date.max()
category_data = extend_dataset(category_data, last_train_date=last_train_date, future_weeks_num=1)

## 📊 Расчёт rolling-признаков (скользящих аггрегаций)

Теперь посчитаем базовые признаки по скользящему окну (rolling window):

- `rolling_sum_during_week` — сколько статей было опубликовано за последние 7 дней.
- `rolling_max_during_week` — максимум статей этой категории в день за последние 7 дней.
- `rolling_min_during_month` и `rolling_max_during_month` — минимум и максимум за последние 28 дней (месяц).

Эти признаки помогут модели понимать недавнюю активность в категории.


In [4]:
def get_rolling_features(cat_data: pd.DataFrame):
    daily_features = pd.DataFrame()
    weekly_rolling = category_data['num_papers'].rolling('7D', min_periods=1)
    daily_features[f'rolling_sum_during_week'] = weekly_rolling.sum()
    daily_features[f'rolling_max_during_week'] = weekly_rolling.max()

    month_rolling = category_data['num_papers'].rolling('28D', min_periods=1)
    daily_features[f'rolling_min_during_month'] = month_rolling.min()
    daily_features[f'rolling_max_during_month'] = month_rolling.max()
    return daily_features

rolling_features = get_rolling_features(cat_data=category_data)
rolling_features

,rolling_sum_during_week,rolling_max_during_week,rolling_min_during_month,rolling_max_during_month
2000-01-01,5.0,5.0,5.0,5.0
2000-01-02,11.0,6.0,5.0,6.0
2000-01-03,25.0,14.0,5.0,14.0
2000-01-04,35.0,14.0,5.0,14.0
2000-01-05,51.0,16.0,5.0,16.0
...,...,...,...,...
2025-02-12,65.0,29.0,5.0,32.0
2025-02-13,36.0,25.0,5.0,32.0
2025-02-14,11.0,6.0,5.0,32.0
2025-02-15,6.0,6.0,5.0,32.0


## ⏪ Добавление лагов (значений сдвига)

Теперь добавим **лаг-признаки** — значения, которые были ровно неделю назад:

- Например, `rolling_sum_during_week_last_week` — это сумма за неделю, но из **предыдущей** недели.
- Такие признаки важны, чтобы модель могла "вспоминать", что происходило ранее, и делать выводы на основе динамики.

Мы используем `.shift(freq='7D')`, чтобы сместить значения на 7 дней назад.

In [5]:
def add_lag_features(features: pd.DataFrame):
    new_features = features.copy()
    for col in features.columns:
        new_features[f'{col}_last_week'] = features[col].shift(freq=pd.Timedelta(days=7))
    return new_features.dropna()

lag_features = add_lag_features(rolling_features)
lag_features

,rolling_sum_during_week,rolling_max_during_week,rolling_min_during_month,rolling_max_during_month,rolling_sum_during_week_last_week,rolling_max_during_week_last_week,rolling_min_during_month_last_week,rolling_max_during_month_last_week
2000-01-08,71.0,16.0,4.0,16.0,5.0,5.0,5.0,5.0
2000-01-09,67.0,16.0,2.0,16.0,11.0,6.0,5.0,6.0
2000-01-10,70.0,17.0,2.0,17.0,25.0,14.0,5.0,14.0
2000-01-11,84.0,24.0,2.0,24.0,35.0,14.0,5.0,14.0
2000-01-12,86.0,24.0,2.0,24.0,51.0,16.0,5.0,16.0
...,...,...,...,...,...,...,...,...
2025-02-11,97.0,32.0,5.0,32.0,134.0,29.0,7.0,31.0
2025-02-12,65.0,29.0,5.0,32.0,137.0,32.0,7.0,32.0
2025-02-13,36.0,25.0,5.0,32.0,138.0,32.0,7.0,32.0
2025-02-14,11.0,6.0,5.0,32.0,149.0,32.0,7.0,32.0


## 📆 Преобразование в недельный уровень

Поскольку нам нужно делать прогноз **по неделям**, агрегируем ежедневные признаки в недельные.

- Каждому дню мы сопоставим конец недели (воскресенье).
- Затем для каждой недели возьмём **последнее доступное значение** признаков.
- Это даст нам ровно одну строку признаков на каждую неделю.

Теперь мы готовы строить таргет и обучать модель.


In [6]:
def build_weekly_features(features):
    daily_features = features.reset_index(names="day")
    daily_features['week'] = daily_features['day'] + pd.to_timedelta(6 - daily_features['day'].dt.weekday, unit='D')
    weekly_features = daily_features.groupby('week').last().reset_index()
    return weekly_features.drop('day', axis=1)

weekly_features = build_weekly_features(lag_features)
weekly_features

,week,rolling_sum_during_week,rolling_max_during_week,rolling_min_during_month,rolling_max_during_month,rolling_sum_during_week_last_week,rolling_max_during_week_last_week,rolling_min_during_month_last_week,rolling_max_during_month_last_week
0,2000-01-09,67.0,16.0,2.0,16.0,11.0,6.0,5.0,6.0
1,2000-01-16,105.0,24.0,2.0,24.0,67.0,16.0,2.0,16.0
2,2000-01-23,102.0,26.0,2.0,26.0,105.0,24.0,2.0,24.0
3,2000-01-30,108.0,24.0,2.0,26.0,102.0,26.0,2.0,26.0
4,2000-02-06,101.0,24.0,2.0,26.0,108.0,24.0,2.0,26.0
...,...,...,...,...,...,...,...,...,...
1306,2025-01-19,145.0,29.0,4.0,47.0,123.0,30.0,4.0,60.0
1307,2025-01-26,157.0,31.0,4.0,47.0,145.0,29.0,4.0,47.0
1308,2025-02-02,139.0,29.0,7.0,31.0,157.0,31.0,4.0,47.0
1309,2025-02-09,143.0,32.0,5.0,32.0,139.0,29.0,7.0,31.0


## 🎯 Построение целевой переменной (target)

Теперь создадим **таргет** — количество публикаций в каждой неделе, сдвинутое на `week_horizon`.

- Если `week_horizon=1`, мы будем предсказывать число публикаций **на следующей неделе**.
- Мы агрегируем данные по неделям (`resample('W')`) и сдвигаем их вверх (`shift(-1)`), чтобы каждая неделя "смотрела в будущее".

Это наша целевая переменная для обучения модели.


In [7]:
def build_targets(category_data, week_horizon: int):
    targets = category_data.resample('W').sum().shift(-week_horizon).num_papers.rename('target')
    targets.index.name = 'week'
    return targets

targets = build_targets(category_data=category_data, week_horizon=1)
targets

week
2000-01-02      67
2000-01-09     105
2000-01-16     102
2000-01-23     108
2000-01-30     101
              ... 
2025-01-19     157
2025-01-26     139
2025-02-02     143
2025-02-09       0
2025-02-16    None
Freq: W-SUN, Name: target, Length: 1312, dtype: object

# Итак по одной категории у нас уже есть готовый датасет для тренировки!

In [8]:
current_data = weekly_features.merge(targets, on='week')
current_data

,week,rolling_sum_during_week,rolling_max_during_week,rolling_min_during_month,rolling_max_during_month,rolling_sum_during_week_last_week,rolling_max_during_week_last_week,rolling_min_during_month_last_week,rolling_max_during_month_last_week,target
0,2000-01-09,67.0,16.0,2.0,16.0,11.0,6.0,5.0,6.0,105
1,2000-01-16,105.0,24.0,2.0,24.0,67.0,16.0,2.0,16.0,102
2,2000-01-23,102.0,26.0,2.0,26.0,105.0,24.0,2.0,24.0,108
3,2000-01-30,108.0,24.0,2.0,26.0,102.0,26.0,2.0,26.0,101
4,2000-02-06,101.0,24.0,2.0,26.0,108.0,24.0,2.0,26.0,96
...,...,...,...,...,...,...,...,...,...,...
1306,2025-01-19,145.0,29.0,4.0,47.0,123.0,30.0,4.0,60.0,157
1307,2025-01-26,157.0,31.0,4.0,47.0,145.0,29.0,4.0,47.0,139
1308,2025-02-02,139.0,29.0,7.0,31.0,157.0,31.0,4.0,47.0,143
1309,2025-02-09,143.0,32.0,5.0,32.0,139.0,29.0,7.0,31.0,0


## 🏗️ Сбор финального датасета для обучения

Теперь объединим всё вместе для всех категорий:

1. Для каждой категории:
   - Извлекаем временной ряд.
   - Добавляем будущую неделю для генерации признаков.
   - Считаем rolling-признаки.
   - Добавляем лаги (предыдущие недели).
   - Агрегируем всё на недельном уровне.
   - Создаём `target` — количество публикаций на следующей неделе.
2. Объединяем данные всех категорий в один финальный `dataset`.
3. Преобразуем колонку `category` в категориальный тип (для LightGBM).

Теперь у нас есть полноценный обучающий набор, готовый для машинного обучения! 💪


In [9]:
from tqdm.auto import tqdm

last_train_date=train.submitted_date.max()
progress_bar = tqdm(train.category.unique())

dataset = []
for category in progress_bar:
    category_data = get_category_data(train, category)
    extended_category_data = extend_dataset(category_data, last_train_date=last_train_date, future_weeks_num=1)
    rolling_features = get_rolling_features(cat_data=extended_category_data)
    lag_features = add_lag_features(rolling_features)
    weekly_features = build_weekly_features(lag_features)
    targets = build_targets(category_data=category_data, week_horizon=1)
    data = weekly_features.merge(targets, on='week')
    data['category'] = category
    dataset.append(data)

dataset = pd.concat(dataset)
dataset['category'] = dataset['category'].astype('category')
dataset

  0%|          | 0/140 [00:00<?, ?it/s]

,week,rolling_sum_during_week,rolling_max_during_week,rolling_min_during_month,rolling_max_during_month,rolling_sum_during_week_last_week,rolling_max_during_week_last_week,rolling_min_during_month_last_week,rolling_max_during_month_last_week,target,category
0,2000-01-09,67.0,16.0,2.0,16.0,11.0,6.0,5.0,6.0,105.0,hep-ph - High Energy Physics - Phenomenology
1,2000-01-16,105.0,24.0,2.0,24.0,67.0,16.0,2.0,16.0,102.0,hep-ph - High Energy Physics - Phenomenology
2,2000-01-23,102.0,26.0,2.0,26.0,105.0,24.0,2.0,24.0,108.0,hep-ph - High Energy Physics - Phenomenology
3,2000-01-30,108.0,24.0,2.0,26.0,102.0,26.0,2.0,26.0,101.0,hep-ph - High Energy Physics - Phenomenology
4,2000-02-06,101.0,24.0,2.0,26.0,108.0,24.0,2.0,26.0,96.0,hep-ph - High Energy Physics - Phenomenology
...,...,...,...,...,...,...,...,...,...,...,...
1305,2025-01-12,62.0,11.0,5.0,24.0,60.0,11.0,5.0,24.0,65.0,q-bio
1306,2025-01-19,65.0,15.0,4.0,15.0,62.0,11.0,5.0,24.0,79.0,q-bio
1307,2025-01-26,79.0,15.0,4.0,15.0,65.0,15.0,4.0,15.0,84.0,q-bio
1308,2025-02-02,84.0,20.0,3.0,20.0,79.0,15.0,4.0,15.0,90.0,q-bio


## 🧪 Разделение на обучающую, валидационную и тестовую выборки

Перед обучением модели нужно правильно разделить данные:

- **train_dataset** — всё до последних 4 недель (обучение).
- **valid_dataset** — последние 4 недели перед последней известной датой (валидация).
- **test_dataset** — данные без таргета (будущее, на которое мы будем делать предсказание).

Это разделение имитирует реальную ситуацию: мы учимся на прошлом, проверяем на последнем известном фрагменте, и делаем прогноз на будущее.


In [10]:
labeled_data = dataset[dataset.target.notnull()].reset_index(drop=True).dropna()

n_valid_weeks = 4
valid_start_date = last_train_date - pd.Timedelta(days=7 * n_valid_weeks)

print(f"Предпоследняя неделя: {valid_start_date}")
valid_dataset = labeled_data[labeled_data.week > valid_start_date]
train_dataset = labeled_data[labeled_data.week < valid_start_date]
test_dataset = dataset[dataset.target.isnull()].reset_index(drop=True)

print(train_dataset.shape, valid_dataset.shape, test_dataset.shape)

Предпоследняя неделя: 2025-01-12 00:00:00
(175355, 11) (420, 11) (140, 11)


## 📦 Подготовка данных для LightGBM

Теперь подготовим данные в нужном формате для обучения модели:

- `train_set` и `valid_set` создаются в формате `lightgbm.Dataset` — это специальный формат для ускорения обучения.
- `test_set` остаётся обычным `DataFrame`, потому что мы просто хотим получить предсказания.

Колонку `target` мы используем только как целевую переменную (`label`).


In [11]:
import lightgbm

train_set = lightgbm.Dataset(train_dataset.drop(['week', 'target'], axis=1), label=train_dataset['target'])
valid_set = lightgbm.Dataset(valid_dataset.drop(['week', 'target'], axis=1), label=valid_dataset['target'])
test_set = test_dataset.drop(['week', 'target'], axis=1)

## 🌲 Обучение модели LightGBM с кастомной метрикой

Теперь мы обучим модель LightGBM для задачи регрессии:

- Используем метрику `Safe MAPE` — она помогает избежать слишком большого штрафа на малых значениях.
- Указываем параметры модели:
  - `objective: regression` — мы предсказываем количество статей (то есть числа).
  - `learning_rate: 0.05` — насколько быстро обучается модель.
  - `depth: 5` — максимальная глубина дерева.
  - `metric: None` — мы используем свою метрику, не встроенную.

Мы также включаем:
- **Раннюю остановку** (`early_stopping`), чтобы не переобучиться.
- **Логгинг** каждые 50 итераций, чтобы отслеживать процесс обучения.

Параметры тренировки были выбраны случайно, просто чтобы натренировать хоть что-то

In [12]:
def safe_mape_lgb(y_pred, dataset):
    y_true = dataset.get_label()
    denominator = pd.Series(y_true).abs().clip(lower=10.0)
    error = abs(y_pred - y_true) / denominator
    return 'safe_mape', error.mean(), False  # False = the lower the better

# 2. Параметры модели
params = {
    'objective': 'regression',
    'learning_rate': 0.05,
    'depth': 5,
    'verbosity': -1,
    'metric': 'None'
}

# 3. Обучение с кастомной метрикой
model = lightgbm.train(
    params,
    train_set,
    num_boost_round=200,
    valid_sets=[valid_set],
    valid_names=['valid'],
    feval=safe_mape_lgb,
    callbacks=[
        lightgbm.early_stopping(stopping_rounds=50),
        lightgbm.log_evaluation(period=50)
    ]
)


Training until validation scores don't improve for 50 rounds
[50]	valid's safe_mape: 0.217514
[100]	valid's safe_mape: 0.198487
[150]	valid's safe_mape: 0.197603
[200]	valid's safe_mape: 0.19758
Did not meet early stopping. Best iteration is:
[186]	valid's safe_mape: 0.197523


# Используем нашу натренированную модель на тестовых данных

In [13]:
test_dataset['predicted'] = model.predict(test_set)
test_dataset[['category', 'predicted']]

,category,predicted
0,hep-ph - High Energy Physics - Phenomenology,136.662937
1,math.CO - Combinatorics,98.995191
2,cs.CG - Computational Geometry,5.956679
3,physics.gen-ph - General Physics,6.885276
4,math.CA - Classical Analysis and ODEs,28.818146
...,...,...
135,eess.IV - Image and Video Processing,86.364232
136,eess.SP - Signal Processing,105.350379
137,q-fin.MF - Mathematical Finance,6.666041
138,cond-mat,425.298719


# Делаем финальный сабмишен

проверяем что наш submission упорядочен также как sample_submission.csv

In [14]:

sample_submission = pd.read_csv(os.path.join(work_dir, "sample_submission.csv"))
sample_submission['category'] = sample_submission['id'].apply(lambda x: x.split('__')[0])
sample_submission = sample_submission.merge(test_dataset[['category', 'predicted']], on='category')
sample_submission[['id', 'predicted']].to_csv('ml_baseline.csv', index=False)